https://github.com/opencv/opencv/tree/master/data/haarcascades

https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_objdetect/py_face_detection/py_face_detection.html


https://docs.opencv.org/2.4/doc/user_guide/ug_traincascade.html

In [ ]:
import numpy as np
import cv2 as cv

rostro = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')

In [ ]:


cap = cv.VideoCapture(0)
x=y=w=h= 0 
img = 0
count = 0
while True:
    ret, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    rostros = rostro.detectMultiScale(gray, 1.3, 5)
    for(x, y, w, h) in rostros:
        m= int(h/2)
        frame = cv.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)
       # frame = cv.rectangle(frame, (x,y+m), (x+w, y+h), (255, 0 ,0), 2 )
        frame2 = frame[y:y+h, x:x+w]
        cv.imshow('rostros2', frame2)
        cv.imwrite('./capturas/johann/'+str(i)+'.png', frame2)
       # img = 180- frame[y:y+h,x:x+w]
       # count = count + 1   

        # !  para que las imágenes tengan el mismo tamaño se debe usar la función resize 100x100 px
    #name = '/home/likcos/imgs/cara'+str(count)+'.jpg'
    #cv.imwrite(name, frame)
    cv.imshow('rostros', frame)
    #cv.imshow('cara', img)
    i = i+1 
    k = cv.waitKey(1)
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()
    

In [ ]:
pwd

# Codigo para realizar la capturas

In [11]:
import cv2 as cv
import os

# Verificar si la carpeta de destino existe, si no, crearla
output_dir = './capturas/johann/neutro/'
if not os.path.exists(output_dir):
    print('No existe el directorio')
    

# Cargar el clasificador de Haar para la detección de rostros
rostro = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')

# Iniciar la captura de vídeo desde la cámara
cap = cv.VideoCapture(0)

# Contador para el nombre de los archivos de salida
i = 232

while True:
    # Capturar un fotograma de la cámara
    ret, frame = cap.read()

    # Convertir el fotograma a escala de grises
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    # Detectar rostros en la imagen en escala de grises
    rostros = rostro.detectMultiScale(gray, 1.3, 5)
    
    # Iterar sobre cada rostro detectado
    for(x, y, w, h) in rostros:
        # Dibujar un rectángulo alrededor del rostro detectado en el fotograma original
        
        # Dibujar un rectángulo alrededor del rostro detectado en el fotograma original
        frame = cv.rectangle(frame, (x-20, y-20), (x+w+20, y+h+20), (0, 255, 0), 2)

        # frame = cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Recortar el área del rostro detectado
        frame2 = frame[y:y+h, x:x+w]
        
        # Redimensionar el área del rostro a 100x100 píxeles
        frame2 = cv.resize(frame2, (100, 100), interpolation=cv.INTER_AREA)
        
        # Guardar el rostro redimensionado como una imagen PNG
        cv.imwrite(output_dir+'johannneutro'+str(i)+'.png', frame2)
        
    # Mostrar el fotograma con los rostros detectados
    cv.imshow('Rostros', frame)
    
    # Incrementar el contador de archivos de salida
    i += 1
    
    # Esperar por la tecla ESC para salir del bucle
    k = cv.waitKey(1)
    if k == 27:
        break

# Liberar los recursos de la cámara y cerrar las ventanas
cap.release()
cv.destroyAllWindows()

# Codigo para generar el xml

In [ ]:
import cv2 as cv 
import numpy as np 
import os

dataSet = 'capturas'
faces  = os.listdir(dataSet)
print(faces)

labels = []
facesData = []
label = 0 
for face in faces:
    facePath = dataSet+'/'+face
    for faceName in os.listdir(facePath):
        labels.append(label)
        facesData.append(cv.imread(facePath+'/'+faceName,0))
    label = label + 1
print(np.count_nonzero(np.array(labels)==0)) 

faceRecognizer = cv.face.EigenFaceRecognizer_create()
faceRecognizer.train(facesData, np.array(labels))
faceRecognizer.write('laloEigenface.xml')


# Codigo para cargar el xml

In [ ]:
import cv2 as cv
import os

# Verify that the Haar cascade file exists
cascade_path = 'haarcascade_frontalface_alt.xml'
if not os.path.isfile(cascade_path):
    raise FileNotFoundError(f"The Haar cascade file '{cascade_path}' was not found. Please check the path.")

# Initialize face recognizer
faceRecognizer = cv.face.LBPHFaceRecognizer_create()

# Read the trained model
model_path = "./EmocionesLBPHZwei.xml"
if not os.path.isfile(model_path):
    raise FileNotFoundError(f"The model file '{model_path}' was not found. Please check the path.")
faceRecognizer.read(model_path)

# Define emotion labels
faces = ["enojo", "feliz", "sorpresa"]

# Initialize video capture
cap = cv.VideoCapture(0)
if not cap.isOpened():
    raise Exception("Could not open video device")

# Load the Haar cascade for face detection
rostro = cv.CascadeClassifier(cascade_path)
if rostro.empty():
    raise Exception("Failed to load Haar cascade file")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break
    
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cpGray = gray.copy()
    rostros = rostro.detectMultiScale(gray, 1.3, 3)
    
    for (x, y, w, h) in rostros:
        frame2 = cpGray[y:y+h, x:x+w]
        frame2 = cv.resize(frame2, (100, 100), interpolation=cv.INTER_CUBIC)
        result = faceRecognizer.predict(frame2)
        
        # Display the prediction result
        if result[1] < 100:
            cv.putText(frame, f'{faces[result[0]]}', (x, y-25), 2, 1.1, (0, 255, 0), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        else:
            cv.putText(frame, 'Desconocido', (x, y-20), 2, 0.8, (0, 0, 255), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
    
    cv.imshow('frame', frame)
    if cv.waitKey(1) == 27:  # Press 'ESC' to exit
        break

cap.release()
cv.destroyAllWindows()